In [16]:
import re
import pandas as pd
import numpy as np
import os, shutil
import inspect
import datetime as dt
from sys import platform
from glob import glob
from tqdm import tqdm
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

import seronetDataclass as seroClass
import seronetFunctions as seroFxn

In [17]:
cloud = '/Users/liualg/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/Curation channel/ImmPort Uploads/'
PMID_LIST = pd.read_excel(os.path.join(cloud,'Immport-DR46 (November).xlsx'))['PMID (link to Box)'][1:]
PMID_LIST = [
    35180044, 35390296, 34368647, 35284808, 35483404, 34546094, 34250518, 34320281, 35085183, 35061630, 35704428, 33688034, 35756977, 34150933, 35090596, 35932763, 35839768, 35081612, 34383889, 36058184, 35576468, 34308390, 35881010, 35881005
]

In [18]:
# finding correct Box Base
if platform == "darwin":
    box_base = "~/Library/CloudStorage/Box-Box/SeroNet Public Data"
else: 
    print("User has windows")
    box_base = os.path.join("Users",os.getlogin(), "Box")


In [19]:
def get_all_files(pmid):
    if isinstance(pmid, int):
        pmid = str(pmid)
    else:
        pass
    #File Paths
    BASE_DIR = seroFxn.get_box_dir(box_base, pmid)
    OUT_DIR = glob(os.path.join(BASE_DIR,'ImmPort*'))
    OUT_DIR.sort()
    
    for file in glob(os.path.join(OUT_DIR[-1],"*.txt")):
        if 'protocol.txt' in file:
            protocols.append(pd.read_csv(file,sep='\t',header=2).fillna('-'))

#         elif 'experiments' in file:
#             experiments.append(pd.read_csv(file,sep='\t',header=2).fillna('-'))

        elif 'reagent' in file:
            reagents.append(pd.read_csv(file,sep='\t',header=2).fillna('-'))

        elif 'panel' in file:
            assessments.append(pd.read_csv(file,sep='\t',header=2).fillna('-'))

        elif 'subject_human' in file:
            subject_humans.append(pd.read_csv(file,sep='\t',header=2).fillna('-'))

        elif 'subject_organism' in file:
            subject_organisms.append(pd.read_csv(file,sep='\t',header=2).fillna('-'))

        elif 'experiment_samples' in file:
            experiment_samples.append(pd.read_csv(file,sep='\t',header=2).fillna('-'))

        elif 'treatment' in file:
            treatments.append(pd.read_csv(file,sep='\t',header=2).fillna('-'))

        elif 'basic' in file:
            basic_stdy_templates.append(file)
        else:
#             print(OUT_DIR[-1])
            pass

#     return {'basic_stdy_templates':basic_stdy_templates, 'protocols': protocols, 'experiments': experiments,
#             'reagents': reagents,'assessments': assessments,'subject_humans': subject_humans,
#             'subject_organisms': subject_organisms, 'experiment_samples': experiment_samples,
#             'treatments': treatments
#            }

In [285]:
def get_sections(ws, class_names):
    temp = []
    for i in range(1,ws.max_row):
        if ws["A"][i].value in class_names:
            temp.append(i+1)
    return temp

def replace_header(dataframe):
    dataframe.columns = dataframe.iloc[0]
    return dataframe[1:]

# def clean_df(dataframe, add_PMID = False):
    
#     dataframe = seroFxn.edit_df(dataframe)
#     dataframe = dataframe.dropna(how='all',axis=0).dropna(how='all',axis=1)
#     if add_PMID:
#         dataframe['PMID'] = [PMID]*len(dataframe)
#     else:
#         pass
    
#     reut
# #             df['PMID'] = [PMID]*len(df) #LIU
# #             CAT.append(df)



#     dataframe = seroFxn.edit_df(dataframe)
#     dataframe = dataframe.dropna(how='all',axis=0).dropna(how='all',axis=1)
#     if add_PMID:
#         dataframe['PMID'] = [PMID]*len(dataframe)
    
#     return replace_header(dataframe.reset_index())

def get_basic_info(ws, PMID):
    # Class names
    class_names = ['study', 'study_personnel', 'study_categorization', 'study_2_condition_or_disease',
                       'arm_or_cohort', 'study_personnel', 'planned_visit',
                       'inclusion_exclusion', 'study_2_protocol', 'study_file',
                       'study_link','study_pubmed'
                  ]



    VARS_TO_CLEAN = ['', 'N/A', 'n/a', 'na', np.nan, None]
    clean_other = VARS_TO_CLEAN + ['Other']

    ws.delete_cols(1)  
    sp = get_sections(ws, class_names)
    sp.append(200)
    
    for section_number in range(len(sp)-1):
        temp_wb = openpyxl.Workbook()
        temp_ws = temp_wb.active

        #making a temp workbook to store each section. This will be turned into df
        for i in ws.iter_rows(values_only = True,
                                    min_row = sp[section_number]+1,
                                    max_row = sp[section_number+1]-1):
            temp_ws.append(i)
            
#         max_row = temp_ws.max_row
#         max_col = temp_ws.max_column
        seroFxn.remove_excess(temp_ws)
        
        df = pd.DataFrame(temp_ws.values)
        sub_section = ws.cell(row=sp[section_number], column = 1).value.strip()
#         print(sub_section)
        if sub_section == 'study':
            df = seroFxn.edit_df(df)
            df = df.dropna(how='all')
            STUDY.append(df)
            
        elif sub_section == 'study_categorization':
            df = seroFxn.edit_df(df)
            df = df.dropna(how='all',axis=0).dropna(how='all',axis=1)
            df['PMID'] = [PMID]*len(df) #LIU
            CAT.append(df)
            
        elif sub_section == 'study_2_condition_or_disease':
            df = seroFxn.edit_df(df)
            df = df.dropna(how='all',axis=0).dropna(how='all',axis=1)
            df['PMID'] = [PMID]*len(df)
            COD.append(df)
            
        elif sub_section == 'inclusion_exclusion':
            df = seroFxn.edit_df(df).T
            df = df.dropna(how='all',axis=0).dropna(how='all',axis=1)
            IE.append(replace_header(df.reset_index()))
            
        elif sub_section == 'arm_or_cohort':
            df = seroFxn.edit_df(df).T
            df = df.dropna(how='all',axis=0).dropna(how='all',axis=1)
            AOC.append(replace_header(df.reset_index()))
            
        elif sub_section == 'study_personnel':
            df = seroFxn.edit_df(df).T
            df = df.dropna(how='all').dropna(how='all',axis=1)
            PERSONNEL.append(replace_header(df.reset_index()))
            
        elif sub_section == 'planned_visit':
            df = seroFxn.edit_df(df).T
            df = df.dropna(how='all').dropna(how='all',axis=1)
            PV.append(replace_header(df.reset_index()))
            
        elif sub_section == 'study_file':
            df = seroFxn.edit_df(df).T
            df = df.dropna(how='all').dropna(how='all',axis=1)
            FILE.append(replace_header(df.reset_index()))
            
        elif sub_section == 'study_link':
            df = seroFxn.edit_df(df).T
            if df.empty:
                pass
            else:
                df = df.dropna(how='all').dropna(how='all',axis=1)
                df = replace_header(df)
                df['PMID'] = [PMID]*len(df)
                LINK.append(df.reset_index())
            
        else:
            pass
        
#     return {
#      'STUDY':STUDY,'CAT':CAT,'COD':COD,'IE':IE,'AOC':AOC,'PERSONNEL' :PERSONNEL,'PV' :PV,'FILE' :FILE, 'LINK' :LINK
#     }


In [286]:
#templates excluding study
basic_stdy_templates = []
protocols = []
experiments = []
reagents = []
assessments = []
subject_humans = []
subject_organisms = []
experiment_samples = []
treatments = []

for PMID in tqdm(PMID_LIST):
    PMID = str(PMID)
    get_all_files(PMID)

# basic_stdy_template_report = []
protocols_report = pd.concat(protocols)
reagent_report = pd.concat(reagents)
assessment_report = pd.concat(assessments)
subject_human_report = pd.concat(subject_humans)
subject_organism_report = pd.concat(subject_organisms)
experiment_sample_report = pd.concat(experiment_samples)
treatment_report = pd.concat(treatments)    

100%|███████████████████████████████████████████| 24/24 [00:02<00:00, 10.82it/s]


In [287]:
#study split
STUDY  = []
CAT = []
COD = []
IE = []
AOC = []
PERSONNEL = []
PV = []
FILE = []
LINK = []

for basic_path in tqdm(basic_stdy_templates):
    basic_wb = openpyxl.Workbook()
    basic_ws = basic_wb.active

    for r in dataframe_to_rows(pd.read_csv(basic_path, sep='\t',header=None)):
        basic_ws.append(r)

    get_basic_info(basic_ws, re.findall(r'\d{8}',basic_path)[0])

100%|███████████████████████████████████████████| 23/23 [00:01<00:00, 12.03it/s]


In [293]:
# basic_stdy_template_report = []

protocols_report = pd.concat(protocols).reset_index(drop=True)
reagent_report = pd.concat(reagents).reset_index(drop=True)
assessment_report = pd.concat(assessments).reset_index(drop=True)
subject_human_report = pd.concat(subject_humans).reset_index(drop=True)
subject_organism_report = pd.concat(subject_organisms).reset_index(drop=True)
experiment_sample_report = pd.concat(experiment_samples).reset_index(drop=True)
treatment_report = pd.concat(treatments).reset_index(drop=True)

STUDY_report = pd.concat(STUDY).dropna(axis=1,how='all').reset_index(drop=True)
COD_report = pd.concat(COD).reset_index(drop=True)
CAT_report = pd.concat(CAT).reset_index(drop=True)
IE_report = pd.concat(IE).reset_index(drop=True)
AOC_report = pd.concat(AOC).reset_index(drop=True)
PERSONNEL_report = pd.concat(PERSONNEL).reset_index(drop=True)
PV_report = pd.concat(PV).reset_index(drop=True)
FILE_report = pd.concat(FILE).reset_index(drop=True)
LINK_report = pd.concat(LINK).reset_index(drop=True)
LINK_report.columns = ['Name','Value','PMID']

In [295]:
LINK_report

,Name,Value,PMID
0,Gene Expression Omnibus (GEO),https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,35483404
1,Gene Expression Omnibus (GEO),https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi...,35483404
2,BioProject,https://www.ncbi.nlm.nih.gov/bioproject/?term=...,34546094
3,SRA,https://www.ncbi.nlm.nih.gov/sra/?term=SRR1431...,34546094
4,SRA,https://www.ncbi.nlm.nih.gov/sra/?term=SRR1431...,34546094
5,GenBank,https://www.ncbi.nlm.nih.gov/nuccore/MN985325,33688034
6,Github,https://github.com/boyercb/covid-gathering-size,36058184
7,Metabolomics Workbench: Raw and analyzed data,https://doi.org/10.21228/M80981,34308390
8,Machine Learning Analysis Code,https://github.com/e-stan/covid_19_analysis,34308390
9,NCBI SRA (PRJNA806767),https://www.ncbi.nlm.nih.gov/bioproject/PRJNA8...,35881005


In [296]:
# os.mkdir("./DR46/database_report/")

In [297]:
protocols_report.to_csv('./DR46/database_report/protocols_report.csv')
reagent_report.to_csv('./DR46/database_report/reagent_report.csv')
assessment_report.to_csv('./DR46/database_report/assessment_report.csv')
subject_human_report.to_csv('./DR46/database_report/subject_human_report.csv')
subject_organism_report.to_csv('./DR46/database_report/subject_organism_report.csv')
experiment_sample_report.to_csv('./DR46/database_report/experiment_sample_report.csv')
treatment_report.to_csv('./DR46/database_report/treatment_report.csv')
STUDY_report.to_csv('./DR46/database_report/STUDY_report.csv')
COD_report.to_csv('./DR46/database_report/COD_report.csv')
CAT_report.to_csv('./DR46/database_report/CAT_report.csv')
IE_report.to_csv('./DR46/database_report/IE_report.csv')
AOC_report.to_csv('./DR46/database_report/AOC_report.csv')
PERSONNEL_report.to_csv('./DR46/database_report/PERSONNEL_report.csv')
PV_report.to_csv('./DR46/database_report/PV_report.csv')
FILE_report.to_csv('./DR46/database_report/FILE_report.csv')
LINK_report.to_csv('./DR46/database_report/LINK_report.csv')

In [316]:
experiment_sample_report

,Column Name,Expsample ID,Biosample ID,Experiment ID,Reagent ID(s),Treatment ID(s),Result File Name,Expsample Name,Expsample Description,Additional Result File Names,...,Subtype,Biosample Name,Biosample Description,Study Time Collected,Study Time Collected Unit,Study Time T0 Event,Study Time T0 Event Specify,Experiment Name,Experiment Description,Measurement Technique
0,-,PMID35180044_expSample-01,PMID35180044_biosampleID-01,PMID35180044_experimentID-01,PMID35180044_reagentID-01,PMID35180044_treatment,ExperimentalDataInStudyFilesTab.txt,-,-,-,...,Blood,-,-,25,Days,Other,Post-vaccination,Multiplex Bead Array Assay,Quantitative,Multiplex Bead Array Assay
1,-,PMID35180044_expSample-02,PMID35180044_biosampleID-01,PMID35180044_experimentID-02,PMID35180044_reagentID-02,PMID35180044_treatment,ExperimentalDataInStudyFilesTab.txt,-,-,-,...,Blood,-,-,50,Days,Other,Post-vaccination,Multiplex Bead Array Assay,Quantitative,Multiplex Bead Array Assay
2,-,PMID35390296_expSample-01,PMID35390296_biosampleID-01,PMID35390296_experimentID-01,PMID35390296_reagentID-01,PMID35390296_treatment,PMID35390296_Data.xlsx,-,A highly sensitive technique for detecting and...,-,...,-,-,-,0,Days,Other,Post-vaccination,ELISA,Quantitative,ELISA
3,-,PMID35390296_expSample-02,PMID35390296_biosampleID-02,PMID35390296_experimentID-02,PMID35390296_reagentID-02,PMID35390296_treatment,PMID35390296_Data.xlsx,-,"A technique for counting, examining and sortin...",-,...,Peripheral Blood Mononuclear Cell (PBMC),-,-,0,Days,Other,Pre-vaccination,Flow Cytometry,Quantitative,Flow Cytometry
4,-,PMID35390296_expSample-03,PMID35390296_biosampleID-02,PMID35390296_experimentID-02,PMID35390296_reagentID-02,PMID35390296_treatment,PMID35390296_Data.xlsx,-,"A technique for counting, examining and sortin...",-,...,Peripheral Blood Mononuclear Cell (PBMC),-,-,0,Days,Other,Pre-vaccination,Flow Cytometry,Quantitative,Flow Cytometry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,-,PMID34383889_expSample-044,PMID34383889_biosampleID-03,PMID34383889_experimentID-06,PMID34383889_reagentID-06,PMID34383889_treatment,PMID34383889_Index Cases_SeroNet.xls,-,A highly sensitive technique for detecting and...,-,...,-,-,-,28,Days,Other,Post-symptom onset,ELISA,-,ELISA
691,-,PMID35576468_expSample-01,PMID35576468_biosampleID-01,PMID35576468_experimentID-01,PMID35576468_reagents_not_curated,PMID35576468_treatment,PMID35576468_PNAS_AABS_Manry_data-sharing_fina...,-,A highly sensitive technique for detecting and...,-,...,-,-,-,0,Days,Other,Other,ELISA,Quantitative,ELISA
692,-,PMID35576468_expSample-02,PMID35576468_biosampleID-01,PMID35576468_experimentID-01,PMID35576468_reagents_not_curated,PMID35576468_treatment,PMID35576468_PNAS_AABS_Manry_data-sharing_fina...,-,A highly sensitive technique for detecting and...,-,...,-,-,-,0,Days,Other,Other,ELISA,Quantitative,ELISA
693,-,PMID34308390_expSample-01,PMID34308390_biosampleID-01,PMID34308390_experimentID-01,PMID34308390_reagents_not_curated,PMID34308390_treatment,ExperimentalDataInStudyFilesTab.txt,-,-,-,...,-,-,-,0,Days,Other,Other,PCR,-,PCR


In [302]:
min(subject_human_report['Min Subject Age'])

0

In [303]:
max(subject_human_report['Min Subject Age'])

38

In [309]:
temp = []
for a in subject_human_report['Race Specify']:
      try:
            temp.append(a.split('|').strip())

SyntaxError: unexpected EOF while parsing (830233207.py, line 4)

In [332]:
import numpy as np 
other = experiment_sample_report
other['numbers'] = range(len(experiment_sample_report))
other['numbers2'] = np.random.rand(len(experiment_sample_report))
other.fillna('')
other.head(2)

# set(experiment_sample_report['Subtype'])

,Column Name,Expsample ID,Biosample ID,Experiment ID,Reagent ID(s),Treatment ID(s),Result File Name,Expsample Name,Expsample Description,Additional Result File Names,...,Biosample Description,Study Time Collected,Study Time Collected Unit,Study Time T0 Event,Study Time T0 Event Specify,Experiment Name,Experiment Description,Measurement Technique,numbers,numbers2
0,-,PMID35180044_expSample-01,PMID35180044_biosampleID-01,PMID35180044_experimentID-01,PMID35180044_reagentID-01,PMID35180044_treatment,ExperimentalDataInStudyFilesTab.txt,-,-,-,...,-,25,Days,Other,Post-vaccination,Multiplex Bead Array Assay,Quantitative,Multiplex Bead Array Assay,0,0.321063
1,-,PMID35180044_expSample-02,PMID35180044_biosampleID-01,PMID35180044_experimentID-02,PMID35180044_reagentID-02,PMID35180044_treatment,ExperimentalDataInStudyFilesTab.txt,-,-,-,...,-,50,Days,Other,Post-vaccination,Multiplex Bead Array Assay,Quantitative,Multiplex Bead Array Assay,1,0.444125


In [340]:
other['AVG_SALARY'] = other.groupby('Subtype').numbers2.transform('mean').droplevel(0)

In [359]:
other.columns

Index(['Column Name', 'Expsample ID', 'Biosample ID', 'Experiment ID',
       'Reagent ID(s)', 'Treatment ID(s)', 'Result File Name',
       'Expsample Name', 'Expsample Description',
       'Additional Result File Names', 'Study ID', 'Protocol ID(s)',
       'Subject ID', 'Planned Visit ID', 'Type', 'Subtype', 'Biosample Name',
       'Biosample Description', 'Study Time Collected',
       'Study Time Collected Unit', 'Study Time T0 Event',
       'Study Time T0 Event Specify', 'Experiment Name',
       'Experiment Description', 'Measurement Technique', 'numbers',
       'numbers2', 'AVG_SALARY'],
      dtype='object')

In [334]:
def f(x):
    a = x.mean()
    return a if isinstance(a, (float, int)) else list(a)

In [367]:
# other.groupby(['Subtype','Experiment Name'], as_index=False).mean()
other['Count'] = other.groupby(['Subtype','Experiment Name'])['numbers2'].transform('sum')
other.drop_duplicates()

,Column Name,Expsample ID,Biosample ID,Experiment ID,Reagent ID(s),Treatment ID(s),Result File Name,Expsample Name,Expsample Description,Additional Result File Names,...,Study Time Collected Unit,Study Time T0 Event,Study Time T0 Event Specify,Experiment Name,Experiment Description,Measurement Technique,numbers,numbers2,AVG_SALARY,Count
0,-,PMID35180044_expSample-01,PMID35180044_biosampleID-01,PMID35180044_experimentID-01,PMID35180044_reagentID-01,PMID35180044_treatment,ExperimentalDataInStudyFilesTab.txt,-,-,-,...,Days,Other,Post-vaccination,Multiplex Bead Array Assay,Quantitative,Multiplex Bead Array Assay,0,0.321063,0.503884,0.765187
1,-,PMID35180044_expSample-02,PMID35180044_biosampleID-01,PMID35180044_experimentID-02,PMID35180044_reagentID-02,PMID35180044_treatment,ExperimentalDataInStudyFilesTab.txt,-,-,-,...,Days,Other,Post-vaccination,Multiplex Bead Array Assay,Quantitative,Multiplex Bead Array Assay,1,0.444125,0.503884,0.765187
2,-,PMID35390296_expSample-01,PMID35390296_biosampleID-01,PMID35390296_experimentID-01,PMID35390296_reagentID-01,PMID35390296_treatment,PMID35390296_Data.xlsx,-,A highly sensitive technique for detecting and...,-,...,Days,Other,Post-vaccination,ELISA,Quantitative,ELISA,2,0.058245,0.511328,149.398268
3,-,PMID35390296_expSample-02,PMID35390296_biosampleID-02,PMID35390296_experimentID-02,PMID35390296_reagentID-02,PMID35390296_treatment,PMID35390296_Data.xlsx,-,"A technique for counting, examining and sortin...",-,...,Days,Other,Pre-vaccination,Flow Cytometry,Quantitative,Flow Cytometry,3,0.463551,0.473871,5.861139
4,-,PMID35390296_expSample-03,PMID35390296_biosampleID-02,PMID35390296_experimentID-02,PMID35390296_reagentID-02,PMID35390296_treatment,PMID35390296_Data.xlsx,-,"A technique for counting, examining and sortin...",-,...,Days,Other,Pre-vaccination,Flow Cytometry,Quantitative,Flow Cytometry,4,0.233477,0.473871,5.861139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,-,PMID34383889_expSample-044,PMID34383889_biosampleID-03,PMID34383889_experimentID-06,PMID34383889_reagentID-06,PMID34383889_treatment,PMID34383889_Index Cases_SeroNet.xls,-,A highly sensitive technique for detecting and...,-,...,Days,Other,Post-symptom onset,ELISA,-,ELISA,690,0.096703,0.511328,149.398268
691,-,PMID35576468_expSample-01,PMID35576468_biosampleID-01,PMID35576468_experimentID-01,PMID35576468_reagents_not_curated,PMID35576468_treatment,PMID35576468_PNAS_AABS_Manry_data-sharing_fina...,-,A highly sensitive technique for detecting and...,-,...,Days,Other,Other,ELISA,Quantitative,ELISA,691,0.146109,0.511328,149.398268
692,-,PMID35576468_expSample-02,PMID35576468_biosampleID-01,PMID35576468_experimentID-01,PMID35576468_reagents_not_curated,PMID35576468_treatment,PMID35576468_PNAS_AABS_Manry_data-sharing_fina...,-,A highly sensitive technique for detecting and...,-,...,Days,Other,Other,ELISA,Quantitative,ELISA,692,0.932758,0.511328,149.398268
693,-,PMID34308390_expSample-01,PMID34308390_biosampleID-01,PMID34308390_experimentID-01,PMID34308390_reagents_not_curated,PMID34308390_treatment,ExperimentalDataInStudyFilesTab.txt,-,-,-,...,Days,Other,Other,PCR,-,PCR,693,0.064767,0.511328,39.729211


In [365]:
other.dtypes

Column Name                      object
Expsample ID                     object
Biosample ID                     object
Experiment ID                    object
Reagent ID(s)                    object
Treatment ID(s)                  object
Result File Name                 object
Expsample Name                   object
Expsample Description            object
Additional Result File Names     object
Study ID                         object
Protocol ID(s)                   object
Subject ID                       object
Planned Visit ID                 object
Type                             object
Subtype                          object
Biosample Name                   object
Biosample Description            object
Study Time Collected              int64
Study Time Collected Unit        object
Study Time T0 Event              object
Study Time T0 Event Specify      object
Experiment Name                  object
Experiment Description           object
Measurement Technique            object
